In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))


from astropy.io import fits
import dill as pickle
from emcee import EnsembleSampler
import numpy as np
import os
import sys
import random

pwd = os.getcwd()
little_things_root_dir = os.path.dirname(os.path.dirname(pwd))
sys.path.append(little_things_root_dir)

from little_things_lib.galaxy_piecewise import Galaxy
from little_things_lib.piecewise_mcmc_fitter import (
    EmceeParameters,
    generate_nwalkers_start_points, 
    lnprob)
from little_things_lib.plotting import plot_posterior_distributions, plot_walker_paths
from datetime import datetime


RAD_PER_ARCSEC = np.pi / (60*60*180)

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt

## Enter parameters for galaxy in cell below

In [3]:
galaxy_name = 'NGC2366'

velocity_error_2d = 10  # [km/s]

DEG_PER_PIXEL = 4.17e-4
DISTANCE = 3400 # kpc

# can leave these as any number for now, not used
LUMINOSITY = 1e8  # solar luminositiy
HI_MASS = 1e8  # solar masses

In [4]:
mcmc_params = EmceeParameters(
    ndim=10,
    nwalkers=20, 
    nburn=1,
    niter=180,
    nthin=5,
    nthreads=4
)


In [5]:
"""
Expect the data to be provided in following naming convention in 'data' directory:

2D observed velocity field FITS file: <galaxy_name>_1mom.fits
Bbarolo fit parameters text file: <galaxy_name>_ring_parameters.txt
Stellar velocity curve: <galaxy_name>_stellar_velocities.txt
Gas velocity_curve: <galaxy_name>_gas_velocities.txt

"""


data_dir = os.path.join(little_things_root_dir, 'data')

observed_2d_vel_field_fits_file = os.path.join(data_dir, f'{galaxy_name}_1mom.fits')
ring_parameters_file = os.path.join(data_dir, f'{galaxy_name}_ring_parameters.txt')

stellar_velocities_file = os.path.join(data_dir, f'{galaxy_name}_stellar_velocities.txt')
gas_velocities_file = os.path.join(data_dir, f'{galaxy_name}_gas_velocities.txt')


In [6]:
radii_arcsec, test_rotation_curve, inclinations, position_angles, x_centers, y_centers , v_systemics = \
    np.loadtxt(ring_parameters_file, usecols=(1,2,4,5,-4,-3,-2)).T

radii_kpc = radii_arcsec * RAD_PER_ARCSEC * DISTANCE
avg_inclination = np.mean(inclinations)
avg_position_angle = np.mean(position_angles)
avg_x_center = np.mean(x_centers)
avg_y_center = np.mean(y_centers)
v_systemic = np.mean(v_systemics)

observed_2d_vel_field = fits.open(observed_2d_vel_field_fits_file)[0].data

stellar_radii, stellar_vel = np.loadtxt(stellar_velocities_file, unpack=True)
gas_radii, gas_vel = np.loadtxt(gas_velocities_file, unpack=True)

# comment out below and uncomment above lines when actually doing a fit
# the lines below are for testing only!
#stellar_radii = np.linspace(0, 7, 41)
#gas_radii = np.linspace(0, 7, 41)
#stellar_vel = np.linspace(0, 1, 41)
#gas_vel = np.linspace(0, 1, 41)

In [7]:
nparams=10
bin_edges=[]
indices=np.linspace(0,len(radii_kpc)-1,nparams+1)
#np.linspace/np.logpsace
for i in indices:
    bin_edges.append(radii_kpc[int(i)])

In [8]:
avg_per_bin=[]
for edge in range(len(bin_edges)):
    if edge==1:
        index0=[[0]]
        index1=np.where(radii_kpc==(bin_edges[edge]))
    elif edge>1:
        index1=np.where(radii_kpc==(bin_edges[edge]))
        index0=np.where(radii_kpc==(bin_edges[edge-1]))
    if edge!=0:
        count=index0[0][0]
        avg=0
        while count<=index1[0][0]:
            avg+=test_rotation_curve[count]/(index1[0][0]-index0[0][0]+1)
            count+=1
        avg_per_bin.append(avg)

bin_edges[0]=bin_edges[0]-.001
bin_edges[-1]=bin_edges[-1]+.001

In [9]:
galaxy = Galaxy(
    distance_to_galaxy=DISTANCE,  # [kpc] Look this up for the galaxy 
    deg_per_pixel=DEG_PER_PIXEL ,  # 'CRDELT1' and 'CRDELT2' in the FITS file header (use absolute value)
    bin_edges=np.array(bin_edges),
    galaxy_name=np.array(galaxy_name),
    vlos_2d_data=observed_2d_vel_field,
    v_error_2d=velocity_error_2d,
    output_dir='output',
    luminosity=LUMINOSITY,
    HI_mass=HI_MASS)

tilted_ring_params = {
    'v_systemic': v_systemic,
    'radii': radii_kpc,   #sets galaxy.radii
    'inclination': inclinations,
    'position_angle': position_angles,
    'x_pix_center': x_centers,
    'y_pix_center': y_centers
}

galaxy.set_tilted_ring_parameters(**tilted_ring_params)

galaxy.interpolate_baryonic_rotation_curve(
    baryon_type='stellar',
    rotation_curve_radii=stellar_radii,
    rotation_curve_velocities=stellar_vel)

galaxy.interpolate_baryonic_rotation_curve(
    baryon_type='gas',
    rotation_curve_radii=gas_radii,
    rotation_curve_velocities=gas_vel)



### The cell below should be modified for the piecewise model

In [10]:
# initialize MCMC start position and bounds
galaxy.set_piecewise_prior_bounds(
    
    bin_edges,
    test_rotation_curve
  
)

start_pos = generate_nwalkers_start_points(

    mcmc_params.nwalkers,
    galaxy,
    avg_per_bin,
    velocity_error_2d
)

In [11]:
# set to False to greatly reduce size of saved output
save_blob = True

# initialize sampler
sampler = EnsembleSampler(
    mcmc_params.nwalkers,
    mcmc_params.ndim, 
    lnprob, 
    args=[galaxy, save_blob], 
    threads=mcmc_params.nthreads)

# burn in 
sampler.run_mcmc(start_pos, N=mcmc_params.nburn)
start_pos_after_burn_in = sampler._last_run_mcmc_result[0]
sampler.reset()

emcee: Exception while calling your likelihood function:
  params: [ 7.18320389 23.99235681 62.4570884  39.09955497 24.15341923 62.10534881
 63.68555198 88.7281119  70.30514521 40.74937757]
  args: [<little_things_lib.galaxy_piecewise.Galaxy object at 0x1a18a55b50>, True]
  kwargs: {}
  exception:


Traceback (most recent call last):
  File "/Users/stephencoffey/.local/lib/python3.7/site-packages/emcee/ensemble.py", line 519, in __call__
    return self.f(x, *self.args, **self.kwargs)
  File "/Users/stephencoffey/Downloads/little-things/little_things_lib/piecewise_mcmc_fitter.py", line 95, in lnprob
    lnl, bb = lnlike(params, galaxy)
  File "/Users/stephencoffey/Downloads/little-things/little_things_lib/piecewise_mcmc_fitter.py", line 75, in lnlike
    chisq, model_2d_field = chisq_2d(galaxy, galaxy.radii, v_m, v_err_const=galaxy.v_error_2d, record_array=True)
  File "/Users/stephencoffey/Downloads/little-things/little_things_lib/piecewise_mcmc_fitter.py", line 121, in chisq_2d
    v_rot=v_rot_1d_model
  File "/Users/stephencoffey/Downloads/little-things/little_things_lib/galaxy_piecewise.py", line 223, in create_2d_velocity_field
    v_field = imputer.fit_transform(v_field)
  File "/Users/stephencoffey/anaconda2/envs/anna_env/lib/python3.7/site-packages/missingpy/knnimpute.py",

emcee: Exception while calling your likelihood function:
  params: [39.37962751 37.5054657  65.05364769 54.11460287 23.47860442 58.60070025
 47.2364371  77.49512518 47.77317803 15.48361667]
  args: [<little_things_lib.galaxy_piecewise.Galaxy object at 0x1a18aa6610>, True]
  kwargs: {}
  exception:


Traceback (most recent call last):
  File "/Users/stephencoffey/.local/lib/python3.7/site-packages/emcee/ensemble.py", line 519, in __call__
    return self.f(x, *self.args, **self.kwargs)
  File "/Users/stephencoffey/Downloads/little-things/little_things_lib/piecewise_mcmc_fitter.py", line 95, in lnprob
    lnl, bb = lnlike(params, galaxy)
  File "/Users/stephencoffey/Downloads/little-things/little_things_lib/piecewise_mcmc_fitter.py", line 75, in lnlike
    chisq, model_2d_field = chisq_2d(galaxy, galaxy.radii, v_m, v_err_const=galaxy.v_error_2d, record_array=True)
  File "/Users/stephencoffey/Downloads/little-things/little_things_lib/piecewise_mcmc_fitter.py", line 121, in chisq_2d
    v_rot=v_rot_1d_model
  File "/Users/stephencoffey/Downloads/little-things/little_things_lib/galaxy_piecewise.py", line 223, in create_2d_velocity_field
    v_field = imputer.fit_transform(v_field)
  File "/Users/stephencoffey/anaconda2/envs/anna_env/lib/python3.7/site-packages/missingpy/knnimpute.py",

emcee: Exception while calling your likelihood function:
  params: [23.62182098 46.4128174  28.19914934 51.14228786 58.25248774 60.27123863
 50.3701473  84.79442387 81.58437641 30.18841933]
  args: [<little_things_lib.galaxy_piecewise.Galaxy object at 0x1a18aa6490>, True]
  kwargs: {}
  exception:


...

emcee: Exception while calling your likelihood function:
  params: [26.6219852  28.96162789 39.21787903 66.26147392 64.29069011 80.82931454
 41.72655468 38.43067168 62.26674634 27.67925433]
  args: [<little_things_lib.galaxy_piecewise.Galaxy object at 0x1a18a55b50>, True]
  kwargs: {}
  exception:


Traceback (most recent call last):
  File "/Users/stephencoffey/.local/lib/python3.7/site-packages/emcee/ensemble.py", line 519, in __call__
    return self.f(x, *self.args, **self.kwargs)
  File "/Users/stephencoffey/Downloads/little-things/little_things_lib/piecewise_mcmc_fitter.py", line 95, in lnprob
    lnl, bb = lnlike(params, galaxy)
  File "/Users/stephencoffey/Downloads/little-things/little_things_lib/piecewise_mcmc_fitter.py", line 75, in lnlike
    chisq, model_2d_field = chisq_2d(galaxy, galaxy.radii, v_m, v_err_const=galaxy.v_error_2d, record_array=True)
  File "/Users/stephencoffey/Downloads/little-things/little_things_lib/piecewise_mcmc_fitter.py", line 121, in chisq_2d
    v_rot=v_rot_1d_model
  File "/Users/stephencoffey/Downloads/little-things/little_things_lib/galaxy_piecewise.py", line 223, in create_2d_velocity_field
    v_field = imputer.fit_transform(v_field)
  File "/Users/stephencoffey/anaconda2/envs/anna_env/lib/python3.7/site-packages/missingpy/knnimpute.py",

emcee: Exception while calling your likelihood function:
  params: [ 7.01470913 37.74701249 60.91481323 73.46146015 77.76561852 77.59193222
 29.19964468 51.60981344 66.10525254 48.72873203]
  args: [<little_things_lib.galaxy_piecewise.Galaxy object at 0x104730f90>, True]
  kwargs: {}
  exception:


Traceback (most recent call last):
  File "/Users/stephencoffey/.local/lib/python3.7/site-packages/emcee/ensemble.py", line 519, in __call__
    return self.f(x, *self.args, **self.kwargs)
  File "/Users/stephencoffey/Downloads/little-things/little_things_lib/piecewise_mcmc_fitter.py", line 95, in lnprob
    lnl, bb = lnlike(params, galaxy)
  File "/Users/stephencoffey/Downloads/little-things/little_things_lib/piecewise_mcmc_fitter.py", line 75, in lnlike
    chisq, model_2d_field = chisq_2d(galaxy, galaxy.radii, v_m, v_err_const=galaxy.v_error_2d, record_array=True)
  File "/Users/stephencoffey/Downloads/little-things/little_things_lib/piecewise_mcmc_fitter.py", line 121, in chisq_2d
    v_rot=v_rot_1d_model
  File "/Users/stephencoffey/Downloads/little-things/little_things_lib/galaxy_piecewise.py", line 223, in create_2d_velocity_field
    v_field = imputer.fit_transform(v_field)
  File "/Users/stephencoffey/anaconda2/envs/anna_env/lib/python3.7/site-packages/missingpy/knnimpute.py",

ValueError: Some column(s) have more than 80.0% missing values

In [ ]:
# this will break up the fitting procedure into smaller chunks of size batch_size and save progress
dateTimeObj = datetime.now()
timestampStr = dateTimeObj.strftime("%d-%b-%Y")

batch_size = 5
mcmc_output = []
for batch in range(mcmc_params.niter // batch_size):
    if batch == 0:
        batch_start = start_pos_after_burn_in
    else:
        batch_start = None
        sampler.pool = temp_pool
    mcmc_output += sampler.run_mcmc(batch_start, batch_size, thin=mcmc_params.nthin)
    temp_pool = sampler.pool
    del sampler.pool
    with open(f'/Volumes/My Passport for Mac/Piecewise pkl/sampler_{timestampStr}.pkl', 'wb') as f:
        pickle.dump(sampler, f)
    with open(f'/Volumes/My Passport for Mac/Piecewise pkl/mcmc_output_{timestampStr}.pkl', 'wb') as f:
        pickle.dump(mcmc_output, f)
    print(f"Done with steps {batch*batch_size} - {(batch+1)*batch_size} out of {mcmc_params.niter}")


#### The two cells below demonstrate how to load your saved results. This is useful in the follow cases:

1) You finished a long MCMC fitting run and want to plot the results without having to redo the whole thing.

2) Your computer crashed before it was done running the previous MCMC fit and you want to restart it from the last saved iteration. In this case you can run the second cell below to continue the MCMC fitting. The results of the restarted run will be saved separately from the results of the previous run. The combined results will also be saved.

In [ ]:
# example of how to load the pickled objects
# change the name of the files to whatever yours were saved as

with open(f'/Volumes/My Passport for Mac/Piecewise pkl/sampler_{timestampStr}.pkl', 'rb') as f:
    saved_sampler = pickle.load(f)
    restart_pos = saved_sampler._last_run_mcmc_result[0]
with open(f'/Volumes/My Passport for Mac/Piecewise pkl/mcmc_output_{timestampStr}.pkl', 'rb') as f:
    saved_mcmc_output = pickle.load(f)

In [ ]:
# example of how to restart the MCMC fit from the last save point 
# assumes you loaded the sampler and mcmc_output from the saved pickles in the example above

restart_sampler = EnsembleSampler(
    mcmc_params.nwalkers,
    mcmc_params.ndim, 
    lnprob, 
    args=[galaxy, save_blob], 
    threads=mcmc_params.nthreads)
restart_mcmc_output = []

dateTimeObj = datetime.now()
timestampStr = dateTimeObj.strftime("%d-%b-%Y")

batch_size = 10
for batch in range(mcmc_params.niter // batch_size):
    if batch==0:
        batch_start = restart_pos
    else:
        batch_start = None
        restart_sampler.pool = temp_pool
    restart_mcmc_output += restart_sampler.run_mcmc(batch_start, batch_size, thin=mcmc_params.nthin)
    temp_pool = restart_sampler.pool
    del sampler.pool
    with open(f'/Volumes/My Passport for Mac/Piecewise pkl/sampler_{timestampStr}.pkl', 'wb') as f:
        pickle.dump(restart_sampler, f)
    with open(f'/Volumes/My Passport for Mac/Piecewise pkl/mcmc_output_{timestampStr}.pkl', 'wb') as f:
        pickle.dump(restart_mcmc_output, f)
    print(f"Done with steps {batch*batch_size} - {(batch+1)*batch_size} out of {mcmc_params.niter}")

    
# this step adds the MCMC results from the restarted run to the ones that were saved from the previous run.
total_mcmc_output = saved_mcmc_output + restart_mcmc_output
    with open(f'/Volumes/My Passport for Mac/Piecewise pkl/total_mcmc_output_{timestampStr}.pkl', 'wb') as f:
        pickle.dump(total_mcmc_output, f)

In [ ]:
#### Plot the posterior distributions and walker paths

# Uncomment below line if using the the sampler loaded from the save file
sampler=saved_sampler

parameter_labels = sorted(list(galaxy.bounds.keys()))

plot_posterior_distributions(sampler, labels=parameter_labels)

plot_walker_paths(
    sampler,
    mcmc_params,
    labels=parameter_labels)

def get_fit_stats(sampler, labels=parameter_labels):
    for i, label in enumerate(labels):
        chain = sampler.chain[:, :, i].flatten()
        print(f"{label}: {np.mean(chain)} +/- {np.std(chain)}")
        
get_fit_stats(sampler)

In [ ]:
#blobs = np.reshape(sampler.blobs, (int(mcmc_params.niter/mcmc_params.nthin) * mcmc_params.nwalkers, 10))
blobs = [] 
for subarr in sampler.blobs:
    blobs += subarr
blobs = np.array(blobs)

rotcurve_index = len(galaxy.bounds) + 1
v_tot = np.mean( blobs[:, rotcurve_index])

plt.plot(radii_kpc, test_rotation_curve, linewidth=2., label="data", color="black")

plt.step(galaxy.bin_edges[:-1], v_tot, where="post", color=black)
plt.plot(galaxy.bin_edges[-2:], [v_tot[-1], v_tot[-1], color=black], label="total, model")

plt.xlabel("r [kpc]")
plt.ylabel("v [km/s]")
plt.legend()